In [106]:
import os
import subprocess
import numpy as np
import pandas as pd
import random
import mplcursors
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [107]:
def run_cpp_program():
    result = subprocess.run(['../build/MyGTSAM'], 
                            capture_output=True, text=True)

    if result.returncode != 0:
        print(f"Error running C++ program: {result.stderr}")
        return np.inf

    return result.stdout.strip().split('\n')

In [108]:
result = np.array(list(map(float, run_cpp_program())))
result_pd = pd.DataFrame(result.reshape(1,-1))
result_pd.columns = ['s', 'm0', 'n0', 'k0', 'h0', 'A1_x', 'A1_y', 'A2_x', 'A2_y', 'A3_x', 'A3_y', 'A4_x', 'A4_y', 'X0_x', 'X0_y']
X0 = [0.605, 1.9 - 0.805]

In [109]:
path = "/home/ali/Github/Kamal_Dataset/seq8"

content = pd.read_csv(os.path.join(path, "stereo/static_poses.csv"), header=None)
Data = content.values
x_GT = Data[:,0]
x_GT += X0[0] - x_GT[0]
y_GT = Data[:,1]
y_GT += X0[1] - y_GT[0]
z_GT = Data[:,2]

print(x_GT)

content = pd.read_csv(os.path.join(path, "encoder/encoder.csv"), header=None)
Data = content.values
delta = Data[:,0]
tsta = Data[:,1]
brk = Data[:,2]

[0.605      0.19942226 0.54603061 0.75214751 0.15562134 0.58941768
 0.62887208 0.43567073 0.50902405 0.44645508 0.50122058 0.304877
 0.64896227 0.58582667 0.26750988 0.60939234 0.2462634  0.58577556
 0.5847968  0.30483246 0.63897784 0.25812496 0.82485584 0.32771535
 0.44432845 0.63427678 0.50075968 0.18176507 0.6489559  0.36980924
 0.56023549]


In [110]:
r_Delta = 0.025
r_TSTA = 0.035
r_Break = 0.035

# A1 = [result_pd['A1_x'].values, result_pd['A1_y'].values]
# A2 = [result_pd['A2_x'].values, result_pd['A2_y'].values]
# A3 = [result_pd['A3_x'].values, result_pd['A3_y'].values]
# A4 = [result_pd['A4_x'].values, result_pd['A4_y'].values]

a = 1.20
b = 1.90

A1 = [0, b]
A2 = [a, b]
A3 = [0, 0]
A4 = [a, 0]

l_0 = np.column_stack((result_pd['m0'] + result_pd['n0'], result_pd['m0'] + 2*result_pd['k0'], result_pd['n0'] + 2*result_pd['h0']))
m = np.sqrt((x_GT - A1[0])**2 + (y_GT - A1[1])**2)
n = np.sqrt((x_GT - A2[0])**2 + (y_GT - A2[1])**2)
k = np.sqrt((x_GT - A3[0])**2 + (y_GT - A3[1])**2)
h = np.sqrt((x_GT - A4[0])**2 + (y_GT - A4[1])**2)
l = np.column_stack((m + n, m + 2*k, n + 2*h))
delta_l = l - l[0]
l_enc = np.column_stack((r_Delta*delta, r_TSTA*tsta, r_Break*brk))

print(np.column_stack((delta_l, l_enc)))

[[ 0.          0.          0.          0.          0.          0.        ]
 [ 0.51458045 -0.69605141  0.58155971 -0.35669062  0.46219945  0.0743958 ]
 [ 0.79992926 -0.45827453 -0.2480298  -0.34865909  0.04939111  0.50768635]
 [ 0.00315366  0.2562804  -0.19900699 -0.03864376 -0.10290053  0.17133874]
 [ 0.53884252 -0.72187104  0.6791804  -0.39340403  0.6057223   0.04312786]
 [ 0.46527039 -0.25992257 -0.20904637 -0.39809214  0.10435216  0.52595943]
 [ 0.36514935 -0.15017897 -0.22900564 -0.3688543   0.21368417  0.3103543 ]
 [ 0.63981703 -0.5611137   0.0173792  -0.39990153  0.48399948  0.12299205]
 [ 0.80604262 -0.52096175 -0.17753162 -0.23745249  0.27043619  0.04987919]
 [ 0.66227267 -0.55671407 -0.01150899  0.10740067  0.10042237 -0.20674076]
 [ 0.56318453 -0.43811633 -0.08752933  0.32720637 -0.07122368 -0.27478003]
 [ 0.45628926 -0.60489836  0.35828358  0.47420657 -0.0960519  -0.37414634]
 [ 0.13279873 -0.00258902 -0.14152816  0.46232146 -0.37332314 -0.07749574]
 [ 0.70958472 -0.36002274